In [1]:
import torch
import torch.nn as nn
import pandas as pd

from pathlib import Path
import numpy as np
import hvplot.pandas
#from diffroute import get_node_idxs

from src.data import Dataset
from src.model import init_model
from src.helpers import training_loop, hydro_metrics, extract_train, extract_test

from diffhydro.utils import SimpleTimeSeriesSampler

In [2]:
from diffhydro import TimeSeriesThDF

In [38]:
from diffhydro import RivTree

def load_data(root, device="cpu", routing_irf="muskingum"):
    ds_root = root / "ono"
    
    g = pd.read_pickle(ds_root / "g.pkl")
    areas = pd.Series({n:g.nodes[n]['catchment_area'] for n in g.nodes})
    D = pd.Series({n:g.nodes[n]['channel_length'] for n in g.nodes})
    U = pd.Series({n:g.nodes[n]['drainage_area'] for n in g.nodes})
    params = pd.DataFrame({"D":D, "U":U})
    g = RivTree(g, param_df=params)
    g.irf_fn=routing_irf
    
    inp = pd.read_pickle(ds_root / "inp.pkl")[g.nodes] #* 12
    lbl = pd.read_pickle(ds_root / "lbl.pkl")[[2982838930]]
    lbl = lbl / lbl.std()
    inp = inp / inp.std().mean()
    
    inp.index = inp.index.tz_localize(None)
    lbl.index = lbl.index.tz_localize(None)

    inp = TimeSeriesThDF.from_pandas(inp)
    lbl = TimeSeriesThDF.from_pandas(lbl)

    areas = torch.from_numpy(areas[g.nodes].values).float().to(device)
    inp, lbl, g, areas = map(lambda x:x.to(device), [inp, lbl, g, areas])
    return inp, lbl, g, areas

In [39]:
N = 10             # Repeat experiment 10 times
lr = .005          # Initial Runoff Learning rate
wd = .001          # LSTM Weight decay
n_epoch = 400      # training epoch
n_iter = 40        # Iteration per epoch
init_window = 100  # LSTM init window

device = "cuda:1" # Device: Assume available to NVIDIA GPU
irf_fns = [ "hayami", 'pure_lag', 'linear_storage', 'nash_cascade', 'muskingum']

In [71]:
from diffhydro import nse_fn

In [72]:
#inp, lbl = trainer.tr_data.sample()

In [73]:
#lbl.values.var()

In [74]:
#nse_fn??

In [76]:
root = Path("./data")

In [77]:
inp, lbl, g, cat_areas = load_data(root, device)

In [78]:
tr_data = SimpleTimeSeriesSampler(inp[:,:"2016"], lbl[:,:"2016"], init_window, 200)
te_data = SimpleTimeSeriesSampler(inp[:,"2017":], lbl[:,"2017":], init_window, 200)

In [79]:
model = RunoffRoutingModel()

In [80]:
trainer = RunoffRoutingModule(model, tr_data, te_data, cat_areas,
                              routing_lr=.0005,
                              routing_wd=.0001,
                              runoff_lr=.0005,
                              runoff_wd=.0001,
                              scheduler=False).to(device)

In [81]:
trainer.learn(g, cat_areas, n_iter=200)

Training:   0%|          | 0/200 [00:00<?, ?it/s]

-0.03012692928314209


Training:   0%|          | 0/200 [00:00<?, ?it/s]

0.6018459796905518


Training:   0%|          | 0/200 [00:00<?, ?it/s]

...

Training:   0%|          | 0/200 [00:00<?, ?it/s]

0.7620832324028015


Training:   0%|          | 0/200 [00:00<?, ?it/s]

0.8356155157089233


(0    -0.030127
 1     0.601846
 2     0.613940
 3     0.624843
 4     0.582710
 5     0.719725
 6     0.759358
 7     0.766043
 8     0.809140
 9     0.849623
 10    0.845767
 11    0.828952
 12    0.728489
 13    0.847811
 14    0.884892
 15    0.791116
 16    0.894845
 17    0.803790
 18    0.762083
 19    0.835616
 dtype: float64,
 0    -1050.578672
 1        0.708147
 2        0.745507
 3        0.842478
 4        0.838953
 5        0.823532
 6        0.892333
 7        0.895035
 8        0.905362
 9        0.921395
 10       0.904562
 11       0.916528
 12       0.911575
 13       0.923738
 14       0.910200
 15       0.931856
 16       0.932959
 17       0.939017
 18       0.907688
 19       0.937784
 dtype: float64)

In [64]:
self = trainer

In [94]:
inp = self.tr_data.x[:,"2010"]
lbl = self.tr_data.y[:,"2010"]
with torch.no_grad():
    out = self.model(inp, g, cat_areas)[lbl.columns]
out.to_pandas().hvplot() * lbl.to_pandas().hvplot() 

:Overlay
   .Curve.I  :Curve   [index]   (2982838930)
   .Curve.II :Curve   [index]   (2982838930)

In [96]:
nse_fn(out.values[...,self.tr_data.init_len:],
     lbl.values[...,self.tr_data.init_len:], 
     ).mean()

tensor(0.5440, device='cuda:1')

In [85]:
lbl.to_pandas().std()

2982838930    0.342231
dtype: float32

In [86]:
inp = self.te_data.x
lbl = self.te_data.y


In [88]:
lbl.to_pandas().std()

2982838930    1.283852
dtype: float32

In [97]:
inp = self.te_data.x
lbl = self.te_data.y
with torch.no_grad():
    out = self.model(inp, g, cat_areas)[lbl.columns]
out.to_pandas().hvplot() * lbl.to_pandas().hvplot() 

:Overlay
   .Curve.I  :Curve   [index]   (2982838930)
   .Curve.II :Curve   [index]   (2982838930)

In [98]:
nse_fn(out.values[...,self.tr_data.init_len:],
     lbl.values[...,self.tr_data.init_len:], 
     ).mean()

tensor(0.5634, device='cuda:1')

In [35]:
inp, lbl = trainer.tr_data.sample()


In [38]:
inp.shape

torch.Size([1, 37, 70101])

In [27]:
self = trainer

In [29]:
inp, lbl = trainer.tr_data.sample()

self = model

inp_format = self.format_runoff_input(inp)
#runoff_mm1  = self.runoff_model(inp_format)
#runoff_mm = self.format_runoff_output(runoff_mm1, inp)

#out = self.model.runoff_model(inp)
#out = out[lbl.columns]#.values[...,self.tr_data.init_len:]

In [33]:
 self.runoff_model??

Signature:      self.runoff_model(*args, **kwargs)
Type:           Runoff
String form:   
Runoff(
  (core): LSTMModel(
    (lstm): LSTM(1, 256, batch_first=True)
    (linear): Linear(in_features=256, out_features=1, bias=True)
    (out_act): Identity()
  )
)
File:           /data_prediction005/SYSTEM/prediction002/home/tristan/workspace/rivers/diffroute_reviews/DiffHydro/diffhydro/runoff/runoff.py
Source:        
class Runoff(nn.Module):
    """
        Wrapper class around diffroute.LTIRouter.
    """
    def __init__(self, **kwargs):
        super().__init__()
        self.core = LSTMModel(**kwargs)

    def forward(self, inp_df: TimeSeriesThDF) -> TimeSeriesThDF:
        y = self.core(inp_df.values) 
        return TimeSeriesThDF(y,
                              columns=inp_df.columns,
                              index=inp_df.index)
Init docstring: Initialize internal Module state, shared by both nn.Module and ScriptModule.

In [34]:
 self.runoff_model.core.forward??

Signature: self.runoff_model.core.forward(x)
Docstring:
Define the computation performed at every call.

Should be overridden by all subclasses.

.. note::
    Although the recipe for forward pass needs to be defined within
    this function, one should call the :class:`Module` instance afterwards
    instead of this since the former takes care of running the
    registered hooks while the latter silently ignores them.
Source:   
    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        out = self.linear(lstm_out)
        return self.out_act(out)
File:      /data_prediction005/SYSTEM/prediction002/home/tristan/workspace/rivers/diffroute_reviews/DiffHydro/diffhydro/runoff/lstm.py
Type:      method

In [31]:
inp.values.is_contiguous()

True

In [32]:
inp_format.values.is_contiguous()

True

In [67]:
runoff_mm.shape

torch.Size([1, 37, 10101])

In [68]:
runoff_mm1.shape

torch.Size([37, 10101, 1])

In [73]:
model.runoff_model.core.linear.bias

Parameter containing:
tensor([-0.0456], device='cuda:1', requires_grad=True)

In [78]:
idx = 2982838930

runoff_mm.to_pandas()[idx].hvplot() * lbl.to_pandas().hvplot()

:Overlay
   .Curve.A_2982838930 :Curve   [index]   (2982838930)
   .Curve.I            :Curve   [index]   (2982838930)

In [305]:
#runoff_mm.to_pandas()

In [300]:
#runoff_mm.forward??

In [301]:
inp.to_pandas()[idx].hvplot()

:Curve   [index]   (3021114309)

In [255]:
inp.to_pandas().sum(1).hvplot()

:Curve   [index]   (0)

In [256]:
out.to_pandas().hvplot() * lbl.to_pandas().hvplot()

:Overlay
   .Curve.I  :Curve   [index]   (2982838930)
   .Curve.II :Curve   [index]   (2982838930)

In [221]:
trainer.train_one_iter??

Signature: trainer.train_one_iter(g, cat_areas)
Docstring:         
Source:   
    def train_one_iter(self, g, cat_areas):
        """
        """
        inp, lbl = self.tr_data.sample()
        out = self.model(inp, g, cat_areas)
        out = out[lbl.columns].values[...,self.tr_data.init_len:]
        nse = nse_fn(out, lbl.values).mean()
        self.opt.zero_grad()
        loss = 1-nse
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 
                                       max_norm=self.clip_grad_norm)

        self.opt.step()
        return nse.item()
File:      /tmp/ipykernel_4033448/3727876033.py
Type:      method

In [144]:
self = model

In [110]:
inp, lbl = trainer.tr_data.sample()

In [111]:
inp_format = self.format_runoff_input(inp)
runoff_mm  = self.runoff_model(inp_format)
runoff_mm = self.format_runoff_output(runoff_mm, inp)
runoff_m3s = mm_to_m3s(runoff_mm, cat_areas)

In [112]:
g.irf_fn = 

In [113]:
self.routing_model(runoff_m3s, g)

TimeSeriesThDF(B=1, n_series=37, length=201, period=—, dtype=torch.float32, device=cuda:1)

In [ ]:
inp_format

### Parameters

### Training

In [209]:
training_loop??

Signature:
training_loop(
    ds,
    model,
    opt,
    n_iter=100,
    n_epoch=80,
    init_window=100,
    scheduler=None,
    clip_grad_norm=None,
)
Source:   
def training_loop(ds, model, opt, 
                  n_iter=100, 
                  n_epoch=80, 
                  init_window=100, 
                  scheduler=None,
                  clip_grad_norm=None):
    """
        Training loop with an optional learning rate scheduler.
    """
    losses, nses, trnses = [],[],[]

    for epoch in range(1, n_epoch):
        for i in tqdm(range(n_iter)):
            x, y = ds.sample()
            
            o = model(x)
            o = o[..., init_window:]
            y = y[..., init_window:]
            
            loss = ((y[:, 0] - o)**2).mean()
            
            opt.zero_grad()
            loss.backward()
            if clip_grad_norm is not None:
                torch.nn.utils.clip_grad_norm_(model.parameters(), 
                                               max_norm=

In [ ]:
ds = Dataset.load_from_folder(dataset, device)
all_results = {}

for irf_fn in irf_fns:
    print(irf_fn)
    res = []
    for i in range(N):
        # Init model and optimizer
        model = init_model(ds.g, 
                           inp_size=ds.X.shape[-1], 
                           dt = .25, 
                           irf_fn=irf_fn, 
                           device=device)
        opt = torch.optim.Adam([
            {'params': model.runoff_model.parameters(), 'lr': lr, 'weight_decay': wd},
            {'params': [model.routing_params], 'lr': 1e-2, 'weight_decay': 0.0}
        ])
        scheduler = torch.optim.lr_scheduler.StepLR(opt, step_size=100, gamma=0.3)

        # Train
        losses = training_loop(ds, model, opt, n_iter=n_iter, 
                               n_epoch=n_epoch, 
                               init_window=init_window,
                               scheduler=scheduler,
                               clip_grad_norm=1.0)

        # Extract and record results
        ytr, otr = extract_train(model, ds, init_window)
        yte, ote = extract_test(model, ds, init_window)
        res.append((model.cpu(), ytr, otr, yte, ote))
        
    all_results[irf_fn]=res

hayami


 10%|█         | 4/40 [00:00<00:07,  4.54it/s]

### Evaluation

In [ ]:
yte = {k: [x[3] for x in v] for k,v in all_results.items()}
ote = {k: [x[4] for x in v] for k,v in all_results.items()}
results = {k:pd.DataFrame([hydro_metrics(out, y) \
                           for out,y in zip(ote[k], yte[k])])
           for k in ote}

In [ ]:
{k:v.std() for k,v in results.items()}

In [ ]:
{k:v.mean() for k,v in results.items()}